importing the libraries


In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


loading the module 2 dataset

In [10]:
df = pd.read_csv("exoplanet_module2_ready.csv")
df.head()
import pandas as pd

df = pd.read_csv("exoplanet_module2_ready.csv")


derfining the target variable

In [11]:
df["habitability_binary"] = (df["habitability_label"] >= 2).astype(int)
df["habitability_binary"].value_counts()


habitability_binary
1    4055
0    1998
Name: count, dtype: int64

In [12]:
df.columns.tolist()


['pl_name',
 'pl_orbper',
 'pl_rade',
 'st_teff',
 'st_rad',
 'st_mass',
 'st_met',
 'sy_dist',
 'habitability_label',
 'habitability_binary']

selecting the features

In [13]:
features = [
    "pl_orbper",
    "pl_rade",
    "st_teff",
    "st_rad",
    "st_mass",
    "st_met",
    "sy_dist"
]

X = df[features]
y = df["habitability_binary"]


training and splitting

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


scaling pipline

In [15]:
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, X.columns)
    ]
)


Module 4 

Random forest calssifier

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        class_weight="balanced"
    ))
])


trainig the model

In [17]:
rf_model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['pl_orbper', 'pl_rade', 'st_teff', 'st_rad', 'st_mass', 'st_met',
       'sy_dist'],
      dtype='object'))])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced',
                                        n_estimators=200, random_state=42))])

Predicting the model

In [18]:
y_pred = rf_model.predict(X_test)
y_prob = rf_model.predict_proba(X_test)[:, 1]


Modle evaluation

In [19]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report
)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Accuracy: 0.9966969446738233
Precision: 0.9950920245398773
Recall: 1.0
F1 Score: 0.997539975399754
ROC-AUC: 0.9994420468557337

Classification Report:

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       400
           1       1.00      1.00      1.00       811

    accuracy                           1.00      1211
   macro avg       1.00      0.99      1.00      1211
weighted avg       1.00      1.00      1.00      1211



In [20]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)




Accuracy: 0.9966969446738233


In [21]:
print(f"Accuracy Percentage: {accuracy*100:.2f}%")


Accuracy Percentage: 99.67%


In [22]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

# Precision, Recall, F1
print(f"Precision: {precision_score(y_test, y_pred):.2f}")
print(f"Recall: {recall_score(y_test, y_pred):.2f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.2f}")

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Full Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 99.67%
Precision: 1.00
Recall: 1.00
F1 Score: 1.00

Confusion Matrix:
[[396   4]
 [  0 811]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       400
           1       1.00      1.00      1.00       811

    accuracy                           1.00      1211
   macro avg       1.00      0.99      1.00      1211
weighted avg       1.00      1.00      1.00      1211



Logistic Regression (comparison model)

In [23]:
from sklearn.linear_model import LogisticRegression

lr_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

lr_model.fit(X_train, y_train)

lr_pred = lr_model.predict(X_test)
lr_prob = lr_model.predict_proba(X_test)[:, 1]

print("Logistic Regression ROC-AUC:", roc_auc_score(y_test, lr_prob))


Logistic Regression ROC-AUC: 0.8536374845869297


Ranking teh exoplanets by habitability

In [24]:
df["habitability_probability"] = rf_model.predict_proba(X)[:, 1]

top_habitable_planets = (
    df[["pl_name", "habitability_probability"]]
    .sort_values(by="habitability_probability", ascending=False)
    .head(10)
)

top_habitable_planets


,pl_name,habitability_probability
6051,GJ 357 c,1.0
6050,HD 94834 b,1.0
6010,HD 213472 b,1.0
6011,HD 27969 b,1.0
6012,HD 111232 c,1.0
6013,HD 75898 c,1.0
6015,HN Peg b,1.0
6002,HD 17674 b,1.0
6003,HD 20794 e,1.0
6004,HD 211810 b,1.0


In [25]:
# Example
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)


RandomForestClassifier()

In [27]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


In [28]:
print(X_train.shape)
print(type(model))


(4842, 7)
<class 'sklearn.ensemble._forest.RandomForestClassifier'>


In [29]:
import os
print(os.getcwd())


c:\Users\ANANDJOEL\Desktop\Exoplanet Project


In [30]:
print(model)
print(scaler)


RandomForestClassifier()
StandardScaler()


In [33]:
os.makedirs("backend", exist_ok=True)


In [34]:
import pickle
import os

print("Current folder:", os.getcwd())

with open("backend/model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("backend/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("model.pkl and scaler.pkl saved successfully")


Current folder: c:\Users\ANANDJOEL\Desktop\Exoplanet Project
model.pkl and scaler.pkl saved successfully


In [35]:
print(os.path.exists("backend/model.pkl"))
print(os.path.exists("backend/scaler.pkl"))


True
True
